# Cuaderno características de zonas de movimiento

In [18]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import folium
import vaex as vx

Preparando datos

In [2]:
#Leyendo GeoJson zonas de movimiento Uber
zonas = gpd.read_file('Proyecto_wd/mexico_city_agebs.geojson')
#Añadiendo centroides de zonas de movimiento
gdf = gpd.GeoDataFrame.from_features(zonas)
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y
gdf.crs = {'init' :'epsg:4326'}
#Normalizando datos del geojson 
gdf['CVE_MUN'] = gdf['CVE_MUN'].astype(np.int64)
gdf['CVE_LOC'] = gdf['CVE_LOC'].astype(np.int64)
gdf['CVE_ENT'] = gdf['CVE_ENT'].astype(np.int64)
gdf['MOVEMENT_ID'] = gdf['MOVEMENT_ID'].astype(np.int64)
#Leyendo y normalizando Base índice de marginación de CONAPO
marginacion = pd.read_csv("Proyecto_wd/Base_marginacion_AGEB_00-10.csv", encoding='latin-1')
marginacion_ageb = marginacion[(marginacion.CVE_ENT == 9) | (marginacion.CVE_ENT == 15)]
marginacion_ageb = marginacion_ageb[(marginacion_ageb.AÑO == 2010)]
marginacion_ageb['AGEB'] = marginacion_ageb['CVE_AGEB'].astype(str).str[-4:]
marginacion_ageb['MUN'] = marginacion_ageb['CVE_AGEB'].astype(str).str[2:-8].astype(np.int64)
marginacion_ageb['LOC'] = marginacion_ageb['CVE_AGEB'].astype(str).str[5:-4].astype(np.int64)
marginacion_ageb['ENT'] = marginacion_ageb['CVE_AGEB'].astype(str).str[:2].astype(np.int64)
#Reorganizando datos marginación
marginacion_urb = marginacion_ageb[['ENT','MUN','NOM_MUN','LOC','AGEB','GMU']]
marginacion_urb.columns = ['CVE_ENT','CVE_MUN','NOM_MUN', 'CVE_LOC','CVE_AGEB','GMU']

/home/edman/.local/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/edman/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fusionando bases Uber y CONAPO

In [3]:
zonas_GMU = pd.merge(gdf, marginacion_urb, on=['CVE_ENT','CVE_MUN','CVE_LOC','CVE_AGEB'], how='left')
zonas_GMU = zonas_GMU.dropna()
#Recodificando GMU a factores, para evitar problema en IMU
mapping = {'Muy bajo': 1, 'Bajo': 2, 'Medio': 3, 'Alto':4, 'Muy alto':5}
zonas_GMU = zonas_GMU.replace({'GMU':mapping})
zonas_GMU

,geometry,CVE_AGEB,CVE_ENT,CVE_LOC,CVE_MUN,DISPLAY_NAME,MOVEMENT_ID,centroid_lon,centroid_lat,NOM_MUN,GMU
82,"POLYGON ((-98.94275 19.30491, -98.94324 19.304...",0688,15,1,122,Movement Zone 83,83,-98.940113,19.306265,Valle de Chalco Solidaridad,4
83,"POLYGON ((-98.94074 19.30872, -98.94133 19.309...",0692,15,1,122,Movement Zone 84,84,-98.938039,19.308805,Valle de Chalco Solidaridad,4
84,"POLYGON ((-98.93386 19.30632, -98.93337 19.303...",0705,15,1,122,Movement Zone 85,85,-98.935420,19.306312,Valle de Chalco Solidaridad,4
85,"POLYGON ((-98.91412 19.28480, -98.91395 19.284...",071A,15,1,122,Movement Zone 86,86,-98.916838,19.285507,Valle de Chalco Solidaridad,4
86,"POLYGON ((-98.91550 19.28089, -98.91594 19.280...",0724,15,1,122,Movement Zone 87,87,-98.917208,19.281279,Valle de Chalco Solidaridad,4
...,...,...,...,...,...,...,...,...,...,...,...
5079,"POLYGON ((-99.09151 19.41745, -99.09007 19.416...",1187,9,1,17,Movement Zone 5080,5080,-99.094223,19.414374,Venustiano Carranza,2
5080,"POLYGON ((-99.10151 19.42064, -99.10194 19.419...",0988,9,1,17,Movement Zone 5081,5081,-99.102226,19.421245,Venustiano Carranza,1
5081,"POLYGON ((-99.10650 19.42172, -99.10527 19.421...",0969,9,1,17,Movement Zone 5082,5082,-99.106460,19.420089,Venustiano Carranza,1
5082,"POLYGON ((-99.10295 19.42001, -99.10327 19.419...",0973,9,1,17,Movement Zone 5083,5083,-99.104732,19.419484,Venustiano Carranza,1


Generando mapa

In [4]:
m = folium.Map(location=[np.median(zonas_GMU['centroid_lat'].tolist()), np.median(zonas_GMU['centroid_lon'].tolist())], tiles='Stamen Toner', zoom_start=12)

Añadiendo cloropleth 

In [5]:
m.choropleth(
    geo_data=zonas,
    name='Zonas de movimiento marginación',
    data=zonas_GMU,
    columns=['DISPLAY_NAME', 'GMU'],
    fill_color='BuPu',
    key_on='feature.properties.DISPLAY_NAME',
    legend_name='GMU'
)

/home/edman/.local/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


Marcadores y control de capas. Guardando como HTML

In [6]:
# Generando HTML, es demasiado pesado para el archivo
#fg = folium.FeatureGroup(name='Info zonas')
#for lat, lon, val, name in zip(zonas_GMU['centroid_lat'].tolist(), zonas_GMU['centroid_lon'].tolist(), zonas_GMU['IMU'].tolist(), zonas_GMU['DISPLAY_NAME'].tolist()):
#    html = f"""
#    <h2>{name}<\h2><br>
#    <h4>IMU: {int(round(val,0))} HUF <\h4>
#    """
#    fg.add_child(folium.Marker(location=[lat, lon], popup=html))

#m.add_child(fg)

folium.LayerControl().add_to(m)

m.save('Proyecto_wd/mapa.html')


Abriendo base de tiempos promedio de viajes de Uber para el 2019

In [19]:
uber_2019 = vx.open('uber_2019.hdf5')
uber_2019_mayo = uber_2019[uber_2019.month == 5]
#uber_2019_mayo = uber_2019_mayo['sourceid','mean_travel_time']
uber_2019_mayo = uber_2019_mayo['dstid','mean_travel_time']

In [20]:
uber_2019_mayo = uber_2019_mayo.to_pandas_df()
uber_2019_mayo.columns = ['MOVEMENT_ID', 'median_travel']
uber_2019_mayo = uber_2019_mayo.groupby(['MOVEMENT_ID'])[['median_travel']].median()

In [21]:
zonas_viaje = pd.merge(zonas_GMU, uber_2019_mayo, on=['MOVEMENT_ID'], how='left')
GMU_1_viaje = zonas_viaje[(zonas_viaje.GMU == 1)]
GMU_5_viaje = zonas_viaje[(zonas_viaje.GMU == 5)]

Nuevo mapa de zonas de movilidad filtradas

In [93]:
e = folium.Map(location=[np.median(GMU_1_viaje['centroid_lat'].tolist()), np.median(GMU_1_viaje['centroid_lon'].tolist())], tiles='Stamen Toner', zoom_start=12)

In [97]:
e.choropleth(
    geo_data=GMU_1_viaje,
    name='Medianas de viajes promedio',
    data=GMU_1_viaje,
    columns=['DISPLAY_NAME', 'median_travel'],
    fill_color='YlGn',
    key_on='feature.properties.DISPLAY_NAME',
    legend_name='median_travel'
)


folium.LayerControl().add_to(e)

e.save('Proyecto_wd/marginacion_muybaja.html')

/home/edman/.local/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [95]:
f = folium.Map(location=[np.median(GMU_5_viaje['centroid_lat'].tolist()), np.median(GMU_5_viaje['centroid_lon'].tolist())], tiles='Stamen Toner', zoom_start=12)

In [96]:
f.choropleth(
    geo_data=GMU_5_viaje,
    name='Medianas de viajes promedio',
    data=GMU_5_viaje,
    columns=['DISPLAY_NAME', 'median_travel'],
    fill_color='YlGn',
    key_on='feature.properties.DISPLAY_NAME',
    legend_name='median_travel'
)


folium.LayerControl().add_to(f)

f.save('Proyecto_wd/marginacion_muyalta.html')

/home/edman/.local/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [22]:
g = folium.Map(location=[np.median(zonas_viaje['centroid_lat'].tolist()), np.median(zonas_viaje['centroid_lon'].tolist())], tiles='Stamen Toner', zoom_start=12)

In [23]:
g.choropleth(
    geo_data=zonas_viaje,
    name='Medianas de viajes promedio',
    data=zonas_viaje,
    columns=['DISPLAY_NAME', 'median_travel'],
    fill_color='YlGn',
    key_on='feature.properties.DISPLAY_NAME',
    legend_name='median_travel'
)


folium.LayerControl().add_to(g)

g.save('Proyecto_wd/movimiento.html')

/home/edman/.local/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
